In [ ]:
import matplotlib.pyplot as plt
import time
from keras.datasets import cifar10
from sklearn import metrics

import matplotlib.pyplot as plt
import numpy as np

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn.multiclass import OneVsOneClassifier

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
RESHAPED = 3072

X_train = X_train.reshape(50000, RESHAPED)
X_test = X_test.reshape(10000, RESHAPED)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

y_train = y_train.flatten()
y_test = y_test.flatten()

In [ ]:
X_train /= 255.
X_test /= 255.

In [ ]:
pca = PCA()
pca.fit_transform(X_train)

In [ ]:
%%time
# Calculating optimal k to have 95% (say) variance 

k = 0
total = sum(pca.explained_variance_)
current_sum = 0

while(current_sum / total < 0.99):
    current_sum += pca.explained_variance_[k]
    k += 1
k

In [ ]:
# applying the optimum value of k and then fit and transforming the training and test set.
# applying the optimum value of k and then fit and transforming the training and test set.
pca = PCA(n_components=k, whiten=True)

X_train_pca = pca.fit_transform(X_train)
var_exp = pca.explained_variance_ratio_
X_test_pca = pca.transform(X_test)
print("Total variance explianed by ",k, "components:", var_exp.sum())

t-Distributed Stochastic Neighbor Embedding (t-SNE) is a non-linear technique for dimensionality reduction that is particularly well suited for the visualization of high-dimensional datasets

In [ ]:
%%time
tsne = TSNE(n_components=2)
X_training_tsne = tsne.fit_transform(X_train_pca)
X_testing_tsne = tsne.fit_transform(X_test_pca)

In [ ]:
from matplotlib import pyplot as plt
plt.figure( figsize=(15,15) )
plt.scatter( X_train_pca[:, 0], X_train_pca[:, 1], c=y_train, cmap='tab10' )
plt.colorbar()
plt.show()

In [ ]:
from matplotlib import pyplot as plt
plt.figure( figsize=(15,15) )
plt.scatter( X_training_tsne[:, 0], X_training_tsne[:, 1], c=y_train, cmap='tab10' )
plt.colorbar()
plt.show()

In [ ]:
%%time
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

svm = OneVsOneClassifier(LinearSVC(max_iter=10000,dual=False,random_state=25),n_jobs=-1)
param_grid = {
    'estimator__C':[0.0001,0.001,0.01,0.1,0.5,0.8,1,10,100]
}

svm_cv = GridSearchCV(svm, param_grid, cv=10)
svm_cv.fit(X_training_tsne, y_train)
print("Best Parameters :", svm_cv.best_params_)
print("Best Score :",svm_cv.best_score_)
print("Train Accuracy:",svm_cv.score(X_training_tsne, y_train))
print("Test Accuracy:",svm_cv.score(X_testing_tsne, y_test))